# Введение

В последующих лекциях обощается все предыдущее и рассмотриваются следующие задачи NLP:
- Классификация токенов  
- Моделирование языка с масками (например, BERT)  
- Обобщение  
- Перевод  
- Предварительное обучение моделированию причинного языка (например, GPT-2)  
- Ответ на вопросы  

# Token classification (PyTorch)

<small>Первое приложение, которое мы рассмотрим, — это классификация токенов. Эта общая задача охватывает любую проблему, которую можно сформулировать как «присвоение метки каждому токену в предложении», например:  
<big>`Распознавание именованных сущностей (NER)`</big>: поиск сущностей (например, лиц, местоположений или организаций) в предложении. Это можно сформулировать как присвоение метки каждому токену, имея один класс на сущность и один класс для «отсутствия сущности».  
<big>`Разметка частей речи (POS)`</big>: отметьте каждое слово в предложении как соответствующее определенной части речи (например, существительное, глагол, прилагательное и т. д.).  
<big>`Разбиение на фрагменты`</big>: поиск токенов, которые принадлежат одной и той же сущности. Эту задачу (`которую можно объединить с POS или NER`) можно сформулировать как присвоение одной метки (обычно B-) любым токенам, которые находятся в начале фрагмента, другой метки (обычно I-) токенам, которые находятся внутри фрагмента, и третьей метки (обычно O) токенам, которые не принадлежат ни одному фрагменту.

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [5]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

### В этом разделе мы настроим модель (`BERT`) на задачу `NER`, которая затем сможет вычислять прогнозы.

Вы можете найти модель, которую мы обучим и загрузим в Hub, а также перепроверить ее прогнозы здесь https://huggingface.co/huggingface-course/bert-finetuned-ner?text=My+name+is+Sylvain+and+I+work+at+Hugging+Face+in+Brooklyn

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [6]:
from huggingface_hub import notebook_login

notebook_login()

## Подготовка данных

Сначала нам нужен набор данных, подходящий для классификации токенов. В этом разделе мы `будем использовать набор данных CoNLL-2003`, который содержит новостные статьи от Reuters.  
💡 Пока ваш набор данных состоит из текстов, разделенных на слова с соответствующими им метками, вы сможете адаптировать процедуры обработки данных, описанные здесь, к своему собственному набору данных. Вернитесь к Главе 5, если вам нужно освежить в памяти, как загружать собственные пользовательские данные в Dataset.predictions здесь.

### Набор данных CoNLL-2003  
Чтобы загрузить набор данных CoNLL-2003, мы используем метод load_dataset() из библиотеки 🤗 Datasets:

In [7]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2003")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

В частности, мы видим, что набор данных содержит метки для трех задач, которые мы упоминали ранее: NER, POS и фрагментация. Большое отличие от других наборов данных заключается в том, что входные тексты представлены не в виде предложений или документов, а в виде списков слов (последний столбец называется токенами, но он содержит слова в том смысле, что это предварительно токенизированные входные данные, которые все еще должны пройти через токенизатор для токенизации подслов).

Давайте посмотрим на первый элемент обучающего набора:

In [5]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

Поскольку мы хотим выполнить распознавание именованных сущностей, мы рассмотрим теги NER:

In [6]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

Это метки как целые числа, готовые к обучению, но они не обязательно полезны, когда мы хотим проверить данные. Как и в случае с классификацией текста, мы можем получить доступ к соответствию между этими целыми числами и именами меток, посмотрев на атрибут features нашего набора данных:

In [9]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

Итак, этот столбец содержит элементы, которые являются последовательностями ClassLabels. Тип элементов последовательности находится в атрибуте feature этого ner_feature, и мы можем получить доступ к списку имен, посмотрев на атрибут names этого feature:

In [10]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

- O означает, что слово не соответствует ни одной сущности.
- B-PER/I-PER означает, что слово соответствует началу/находится внутри сущности человека.
- B-ORG/I-ORG означает, что слово соответствует началу/находится внутри сущности организации.
- B-LOC/I-LOC означает, что слово соответствует началу/находится внутри сущности местоположения.
- B-MISC/I-MISC означает, что слово соответствует началу/находится внутри разной сущности.  

Теперь расшифровка меток, которые мы видели ранее, дает нам следующее:

In [11]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


А для примера смешивания меток B и I, вот что дает нам тот же код для элемента обучающего набора с индексом 4:

```
'Germany \'s representative to the European Union \'s veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .'
'B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O'
```

Как мы видим, сущности, состоящие из двух слов, например «Европейский союз» и «Вернер Цвингманн», присваивают метку B для первого слова и метку I для второго.

## Обработка данных

Как обычно, наши тексты должны быть преобразованы в идентификаторы токенов, прежде чем модель сможет их осмыслить. Как мы видели в Главе 6, большое отличие в случае задач классификации токенов заключается в том, что у нас есть предварительно токенизированные входные данные. К счастью, API токенизатора может справиться с этим довольно легко; нам просто нужно предупредить токенизатор специальным флагом.

Для начала давайте создадим наш объект токенизатора. Как мы уже говорили, мы будем использовать предварительно обученную модель BERT, поэтому начнем с загрузки и кэширования связанного токенизатора:

In [12]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Вы можете заменить model_checkpoint на любую другую модель из Hub или на локальную папку, в которой вы сохранили предварительно обученную модель и токенизатор. Единственное ограничение заключается в том, что токенизатор должен поддерживаться библиотекой 🤗 Tokenizers, поэтому доступна «быстрая» версия. Вы можете увидеть все архитектуры, которые поставляются с быстрой версией, в этой большой таблице, и чтобы проверить, что используемый вами объект токенизатора действительно поддерживается 🤗 Tokenizers, вы можете посмотреть на его атрибут is_fast:

In [13]:
tokenizer.is_fast

True

Чтобы токенизировать предварительно токенизированные входные данные, мы можем использовать наш токенизатор как обычно и просто добавить `is_split_into_words=True:`

In [14]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [15]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Как мы видим, токенизатор добавил специальные токены, используемые моделью ([CLS] в начале и [SEP] в конце), и оставил большинство слов нетронутыми. Однако слово lamb было токенизировано в два подслова, la и ##mb. Это приводит к несоответствию между нашими входными данными и метками: список меток содержит всего 9 элементов, тогда как наши входные данные теперь содержат 12 токенов. Учет специальных токенов прост (мы знаем, что они находятся в начале и в конце), но нам также нужно убедиться, что мы выровняли все метки с правильными словами.

К счастью, поскольку мы используем быстрый токенизатор, у нас есть доступ к суперспособностям 🤗 токенизаторов, что означает, что мы можем легко сопоставить каждый токен с соответствующим ему словом (как показано в Главе 6):

Приложив немного усилий, мы можем расширить наш список меток для соответствия токенам. Первое правило, которое мы применим, заключается в том, что специальные токены получают метку -100. Это связано с тем, что по умолчанию -100 — это индекс, который игнорируется в функции потерь, которую мы будем использовать (перекрестная энтропия). Затем каждый токен получает ту же метку, что и токен, с которого начиналось слово, в котором он находится, поскольку они являются частью одной и той же сущности. Для токенов внутри слова, но не в начале, мы заменяем B- на I- (поскольку токен не начинает сущность):

In [16]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [17]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


Как мы видим, наша функция добавила -100 для двух специальных токенов в начале и в конце, а также новый 0 для нашего слова, которое было разделено на два токена.

Для предварительной обработки всего нашего набора данных нам нужно токенизировать все входные данные и применить align_labels_with_tokens() ко всем меткам. Чтобы воспользоваться скоростью нашего быстрого токенизатора, лучше всего токенизировать много текстов одновременно, поэтому мы напишем функцию, которая обрабатывает список примеров, и используем метод Dataset.map() с опцией batched=True. Единственное отличие от нашего предыдущего примера заключается в том, что функция word_ids() должна получить индекс примера, для которого нам нужны идентификаторы слов, когда входные данные для токенизатора представляют собой списки текстов (или, в нашем случае, список списков слов), поэтому мы добавляем и это:

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

Обратите внимание, что мы пока не дополнили наши входные данные; мы сделаем это позже, при создании пакетов с помощью сортировщика данных.

Теперь мы можем применить всю эту предварительную обработку за один раз к другим разделам нашего набора данных:

In [19]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Мы сделали самую сложную часть! Теперь, когда данные предварительно обработаны, фактическое обучение будет очень похоже на то, что мы делали в Главе 3.

## Тонкая настройка модели с помощью API Trainer

Фактический код, использующий Trainer, будет таким же, как и раньше; единственными изменениями являются способ объединения данных в пакет и функция вычисления метрики.

### Сопоставление данных

Мы не можем просто использовать DataCollatorWithPadding, как в Главе 3, потому что он только дополняет входные данные (идентификаторы входных данных, маска внимания и идентификаторы типов токенов). Здесь наши метки должны быть дополнены точно так же, как и входные данные, чтобы они оставались того же размера, используя -100 в качестве значения, чтобы соответствующие прогнозы игнорировались при вычислении потерь.

Все это выполняется DataCollatorForTokenClassification. Как и DataCollatorWithPadding, он принимает токенизатор, используемый для предварительной обработки входных данных:

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Чтобы проверить это на нескольких образцах, мы можем просто вызвать его на списке примеров из нашего токенизированного обучающего набора:

In [21]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

Давайте сравним это с метками первого и второго элементов в нашем наборе данных:

In [22]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


Как мы видим, второй набор меток был дополнен до длины первого с помощью -100.

## Метрики

Чтобы Trainer вычислял метрику каждую эпоху, нам нужно определить функцию `compute_metrics()`, которая принимает массивы прогнозов и меток и возвращает словарь с именами и значениями меток.  
Традиционная структура, используемая для оценки прогноза классификации токенов, — `seqeval`. Чтобы использовать эту метрику, нам сначала нужно установить библиотеку seqeval:

In [23]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=615f4aaa0db1564a3b6f0497e715a923250b09dde9705d9235375beae9749646
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


We can then load it via the evaluate.load() function

In [24]:
import evaluate

metric = evaluate.load("seqeval")

Эта метрика не ведет себя как стандартная точность: она фактически принимает списки меток как строки, а не целые числа, поэтому нам нужно будет полностью декодировать прогнозы и метки перед передачей их в метрику. Давайте посмотрим, как это работает. Сначала мы получим метки для нашего первого примера обучения:

In [25]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

Затем мы можем создать для них фальшивые прогнозы, просто изменив значение в индексе 2:

In [26]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

Обратите внимание, что метрика принимает список прогнозов (а не только один) и список меток. Вот вывод:

Это отправляет обратно много информации! Мы получаем точность, полноту и оценку F1 для каждой отдельной сущности, а также общую. Для нашего вычисления метрик мы сохраним только общую оценку, но не стесняйтесь настраивать функцию compute_metrics(), чтобы возвращать все метрики, которые вы хотели бы получить в отчете.

Эта функция compute_metrics() сначала берет argmax логитов, чтобы преобразовать их в прогнозы (как обычно, логиты и вероятности находятся в одном порядке, поэтому нам не нужно применять softmax). Затем нам нужно преобразовать как метки, так и прогнозы из целых чисел в строки. Мы удаляем все значения, где метка равна -100, затем передаем результаты в метод metric.compute():

In [27]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

Теперь, когда это сделано, мы почти готовы определить нашего тренера. Нам просто нужна модель для тонкой настройки!

## Определение модели

Поскольку мы работаем над проблемой классификации токенов, мы будем использовать класс `AutoModelForTokenClassification`. Главное, что нужно помнить при определении этой модели, — это передать некоторую информацию о количестве имеющихся у нас меток. Самый простой способ сделать это — передать это число с аргументом `num_labels`, но если мы хотим, чтобы хороший виджет вывода работал так же, как тот, который мы видели в начале этого раздела, лучше вместо этого задать правильные соответствия меток.

Они должны быть заданы двумя словарями, `id2label` и `label2id`, которые содержат сопоставления из ID в метку и наоборот:

In [28]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [29]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [36]:
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

Теперь мы можем просто передать их методу AutoModelForTokenClassification.from_pretrained(), и они будут установлены в конфигурации модели, а затем правильно сохранены и загружены в Hub:

In [30]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


создание модели выдает предупреждение о том, что некоторые веса не были использованы (те, что из головы предварительной подготовки), а некоторые другие веса инициализируются случайным образом (те, что из головы классификации новых токенов), и что эта модель должна быть обучена. Мы сделаем это, но сначала давайте дважды проверим, что наша модель имеет правильное количество меток:

In [31]:
model.config.num_labels

9

⚠️ Если у вас есть модель с неправильным количеством меток, вы получите неясную ошибку при последующем вызове метода Trainer.train() (что-то вроде «CUDA error: device-side assert triggered»). Это основная причина ошибок, о которых сообщают пользователи, поэтому обязательно выполните эту проверку, чтобы подтвердить, что у вас ожидаемое количество меток.

## Тонкая настройка модели

Теперь мы готовы обучить нашу модель! Нам осталось сделать две последние вещи, прежде чем мы определим нашего тренера: войти в Hugging Face и определить наши аргументы обучения. Если вы работаете в блокноте, есть удобная функция, которая поможет вам в этом:

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

(подсоединившись) После этого мы можем определить наши TrainingArguments:

мы задаем некоторые гиперпараметры (например, скорость обучения, количество эпох для обучения и снижение веса) и указываем push_to_hub=True, чтобы указать, что мы хотим сохранить модель и оценить ее в конце каждой эпохи, а также что мы хотим загрузить наши результаты в Model Hub. Обратите внимание, что вы можете указать имя репозитория, в который хотите отправить данные, с помощью аргумента hub_model_id (в частности, вам придется использовать этот аргумент для отправки в организацию). Например, когда мы отправляли модель в организацию huggingface-course, мы добавили hub_model_id="huggingface-course/bert-finetuned-ner" в TrainingArguments. По умолчанию используемый репозиторий будет находиться в вашем пространстве имен и называться в соответствии с заданным вами выходным каталогом, поэтому в нашем случае это будет "sgugger/bert-finetuned-ner".

💡 Если выходной каталог, который вы используете, уже существует, он должен быть локальным клоном репозитория, в который вы хотите отправить данные. Если это не так, вы получите ошибку при определении вашего Trainer и вам нужно будет задать новое имя.

Наконец, мы просто передаем все Trainer и запускаем обучение:

In [32]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",  # Отключает wandb
    # push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-33-ddd9b7f8600f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.075600,0.068573,0.904964,0.932683,0.918614,0.981162
2,0.035900,0.068876,0.929447,0.944463,0.936895,0.984871
3,0.023300,0.062380,0.936290,0.952205,0.944180,0.986328


TrainOutput(global_step=5268, training_loss=0.06629017282817612, metrics={'train_runtime': 607.3416, 'train_samples_per_second': 69.356, 'train_steps_per_second': 8.674, 'total_flos': 920771584279074.0, 'train_loss': 0.06629017282817612, 'epoch': 3.0})

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
file_path = "/content/drive/My Drive/Hugging_face/bert-finetuned-ner"

In [36]:
trainer.save_model(file_path)
tokenizer.save_pretrained(file_path)

('/content/drive/My Drive/Hugging_face/bert-finetuned-ner/tokenizer_config.json',
 '/content/drive/My Drive/Hugging_face/bert-finetuned-ner/special_tokens_map.json',
 '/content/drive/My Drive/Hugging_face/bert-finetuned-ner/vocab.txt',
 '/content/drive/My Drive/Hugging_face/bert-finetuned-ner/added_tokens.json',
 '/content/drive/My Drive/Hugging_face/bert-finetuned-ner/tokenizer.json')

Тренер также составляет карту модели со всеми результатами оценки и загружает ее.

In [ ]:
# trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/sgugger/bert-finetuned-ner/commit/26ab21e5b1568f9afeccdaed2d8715f571d786ed'

## Пользовательский цикл обучения

Давайте теперь рассмотрим полный цикл обучения, чтобы вы могли легко настроить нужные вам части.

### Подготовка всего для обучения

Сначала нам нужно построить DataLoaders из наших наборов данных. Мы повторно используем наш data_collator как collate_fn и перетасуем обучающий набор, но не проверочный набор:

In [37]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

Далее мы повторно создаем нашу модель, чтобы убедиться, что мы не продолжаем тонкую настройку, начатую ранее, а снова начинаем с предварительно обученной модели BERT:

In [38]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Тогда нам понадобится оптимизатор. Мы будем использовать классический AdamW, который похож на Adam, но с исправлением в способе применения снижения веса:

In [39]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

Получив все эти объекты, мы можем отправить их в метод accelerator.prepare():

In [40]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

🚨 Если вы обучаетесь на TPU, вам нужно будет переместить весь код, начиная с ячейки выше, в специальную функцию обучения. Подробнее см. в Главе 3.

Теперь, когда мы отправили наш train_dataloader в accelerator.prepare(), мы можем использовать его длину для вычисления количества шагов обучения. Помните, что мы всегда должны делать это после подготовки dataloader, так как этот метод изменит его длину. Мы используем классический линейный график от скорости обучения до 0:

In [41]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

Наконец, чтобы отправить нашу модель в Hub, нам нужно будет создать объект Repository в рабочей папке. Сначала войдите в Hugging Face, если вы еще не вошли в систему. Мы определим имя репозитория из идентификатора модели, который мы хотим дать нашей модели (не стесняйтесь заменить repo_name на свой собственный выбор; он просто должен содержать ваше имя пользователя, что и делает функция get_full_repo_name()):

In [42]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'Yurkmez/bert-finetuned-ner-accelerate'

In [44]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Yurkmez/bert-finetuned-ner-accelerate into local empty directory.


## Цикл обучения

Теперь мы готовы написать полный цикл обучения. Чтобы упростить его оценочную часть, мы определяем эту функцию postprocess(), которая принимает прогнозы и метки и преобразует их в списки строк, как ожидает наш объект метрики:

In [45]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

Затем мы можем написать цикл обучения. После определения индикатора выполнения, чтобы отслеживать, как проходит обучение, цикл состоит из трех частей:

Само обучение, которое является классической итерацией по train_dataloader, прямой проход по модели, затем обратный проход и шаг оптимизатора.
Оценка, в которой есть новшество после получения выходных данных нашей модели в пакете: поскольку два процесса могли дополнить входные данные и метки до разных форм, нам нужно использовать accelerator.pad_across_processes(), чтобы сделать прогнозы и метки одинаковой формы перед вызовом метода gather(). Если мы этого не сделаем, оценка либо выдаст ошибку, либо зависнет навсегда. Затем мы отправляем результаты в metric.add_batch() и вызываем metric.compute() после завершения цикла оценки.
Сохранение и загрузка, где мы сначала сохраняем модель и токенизатор, затем вызываем repo.push_to_hub(). Обратите внимание, что мы используем аргумент blocking=False, чтобы сообщить библиотеке Hub 🤗 о необходимости запустить асинхронный процесс. Таким образом, обучение продолжается нормально, а эта (длинная) инструкция выполняется в фоновом режиме.
Вот полный код для цикла обучения:

In [46]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch 0: {'precision': 0.9347021204981488, 'recall': 0.9038242473555737, 'f1': 0.9190038884752213, 'accuracy': 0.9812798022016836}
epoch 1: {'precision': 0.9404240996297543, 'recall': 0.918625678119349, 'f1': 0.9293970893970893, 'accuracy': 0.9833107670571614}


Several commits (2) will be pushed upstream.


epoch 2: {'precision': 0.9488387748232918, 'recall': 0.9292895994725564, 'f1': 0.9389624448330418, 'accuracy': 0.9858568316948254}


Several commits (3) will be pushed upstream.


Если вы впервые видите модель, сохраненную с помощью 🤗 Accelerate, давайте уделим немного времени изучению трех строк кода, которые ее сопровождают:

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

Первая строка говорит сама за себя: она сообщает всем процессам подождать, пока все не окажутся на этом этапе, прежде чем продолжить. Это нужно для того, чтобы убедиться, что у нас одна и та же модель в каждом процессе перед сохранением. Затем мы берем unwrapped_model, которая является базовой моделью, которую мы определили. Метод accelerator.prepare() изменяет модель для работы в распределенном обучении, поэтому у нее больше не будет метода save_pretrained(); метод accelerator.unwrap_model() отменяет этот шаг. Наконец, мы вызываем save_pretrained(), но сообщаем этому методу использовать accelerator.save() вместо torch.save().

После этого у вас должна быть модель, которая выдает результаты, очень похожие на ту, которая была обучена с помощью Trainer. Вы можете проверить модель, которую мы обучили с помощью этого кода, на huggingface-course/bert-finetuned-ner-accelerate. И если вы хотите протестировать какие-либо изменения в цикле обучения, вы можете напрямую реализовать их, отредактировав код, показанный выше!

In [48]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.9988506,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.96476245,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9986118,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]